# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     |Nicolás Martínez Gutiérrez   |
| **Fecha**      11/09/2025|   |
| **Expediente** 751746|   |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

## Regresión Lineal Sin Penalización

In [103]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
import statsmodels.api as sm

In [104]:
data = pd.read_csv("Advertising.csv")

In [105]:
x = data[["TV", "radio", "newspaper"]]
y = data["sales"]

In [106]:
lr = LinearRegression()

In [107]:
lr.fit(x, y)

LinearRegression()

In [108]:
y_pred = lr.predict(x)
r2_score(y, y_pred)

0.8972106381789522

In [109]:
n = len(y)

In [110]:
unos = np.ones([n,1])

In [111]:
X = np.column_stack([unos,x])

In [112]:
ols = sm.OLS(y, X)

In [113]:
results = ols.fit()

In [114]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Mon, 08 Sep 2025   Prob (F-statistic):           1.58e-96
Time:                        19:47:31   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
x1             0.0458      0.001     32.809      0.000       0.043       0.049
x2             0.1885      0.009     21.893      0.000       0.172       0.206
x3            -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Con Penalización (L2)

In [115]:
l2 = Ridge()  
l2.fit(x, y)

Ridge()

In [116]:
y_pred = l2.predict(x)

In [117]:
r2_score(y, y_pred)

0.8972106380074802

In [118]:
l2.intercept_

np.float64(2.9389674583301524)

In [119]:
l2.coef_

array([ 0.04576464,  0.1885251 , -0.00103629])

### RSS

In [120]:
rss = np.sum((y - y_pred) ** 2)

### RSE

In [121]:
n, p = x.shape
rse = np.sqrt(rss / (n - p - 1))

### Varianza de betas

In [122]:
X_ = np.column_stack([np.ones(n), x])
xtx_inv = np.linalg.inv(X_.T @ X_)
var_beta = xtx_inv * (rse ** 2)

### Std de betas

In [123]:
std_beta = np.sqrt(np.diag(var_beta))

### Estadístico t

In [124]:
beta_hat = np.r_[getattr(l2, "intercept_", 0.0), l2.coef_]
t_stats = beta_hat / std_beta

### P-values 

In [125]:
df = n - p - 1
p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df=df))

In [126]:
p_values

array([0.        , 0.        , 0.        , 0.86007527])

## Con Penalización (L1)

In [127]:
l1 = Lasso()   
l1.fit(x, y)

Lasso()

In [128]:
y_pred = l1.predict(x)

In [129]:
r2_score(y, y_pred)

0.8970235728389689

In [130]:
l1.intercept_

np.float64(3.040215583480377)

In [131]:
l1.coef_

array([0.04566142, 0.1834644 , 0.        ])

In [132]:
rss = np.sum((y - y_pred) ** 2)

In [133]:
n, p = x.shape
rse = np.sqrt(rss / (n - p - 1))

In [134]:
X_ = np.column_stack([np.ones(n), x])
xtx_inv = np.linalg.inv(X_.T @ X_)
var_beta = xtx_inv * (rse ** 2)

In [135]:
std_beta = np.sqrt(np.diag(var_beta))

In [136]:
beta_hat = np.r_[getattr(l1, "intercept_", 0.0), l1.coef_]
t_stats = beta_hat / std_beta

In [137]:
df = n - p - 1
p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df=df))

In [138]:
p_values

array([0., 0., 0., 1.])

## Conclusión

En la regresión lineal sin penalización se obtuvo un R2 de 0.897, donde las variables TV y radio resultaron significativas con p-values de 0, mientras que newspaper no lo fue con un p-value de 0.86. Al aplicar penalización L2, el R2 se mantuvo casi igual en 0.8971, con un intercepto de 2.9390 y coeficientes de 0.0458 para TV, 0.1885 para radio y -0.0010 para newspaper, aquí nuevamente TV y radio fueron significativos, pero newspaper tuvo un p-value de 0.8601, confirmando que no aporta al modelo. Finalmente, con penalización L1, el R2 también fue muy similar en 0.897, con un intercepto de 3.0402 y coeficientes de 0.0457 para TV, 0.1837 para radio y 0.0 para newspaper, en este caso, el método eliminó a newspaper del modelo con p-value igual a 1.0, quedando solo TV y radio como variables relevantes. Podemos concluir que la penalización Ridge (L2) reduce el tamaño de los coeficientes, pero no elimina variables. Mientras que, la penalización Lasso (L1) puede llevar algunos coeficientes a cero, eliminando las variables menos relevantes.